In [195]:
# read mat file
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue
import cv2 as cv
import copy
import time
import dlib
from scipy import signal

## 1. Take one data as an example

In [196]:
# load mat file
sample_file = sio.loadmat(r'/Users/zihan/Documents/hku/aiot/group proj/mmpd/p6_0.mat')
print(sample_file.keys())

dict_keys(['__header__', '__version__', '__globals__', 'video', 'GT_ppg', 'light', 'motion', 'exercise', 'skin_color', 'gender', 'glasser', 'hair_cover', 'makeup'])


You can see there are several keys in the data. You will mainly use 'video' and 'GT_ppg' for your evaluation. The 'video' records the stationary scene of the face, and the 'GT_ppg' records the ground truth heart rate of the participants in the video.

In [197]:
video = sample_file['video']
gt = sample_file['GT_ppg']
video.shape, gt.shape

((1800, 80, 60, 3), (1, 1800))

As mentioned in the paper, the sampling rate of the video and ground truth is **30 fps**.

## 2. Evaluation

In [198]:
import os

# Define the directory path
dir_path = '/Users/zihan/Documents/hku/aiot/group proj/mmpd'

# Loop over all files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(dir_path, filename)):
        # Do something with the file
        print(filename)

.DS_Store
p29_0.mat
p29_4.mat
p6_8.mat
p6_4.mat
p29_8.mat
p6_0.mat
p29_16.mat
test.ipynb
p6_16.mat


You need to evaluate the performance of your algorithm on the above eight videos. The evaluation metric is the **mean absolute error (MAE)** between the estimated heart rate and the ground truth. The MAE is calculated as follows:
$$
MAE = \frac{1}{N}\sum_{i=1}^{N}|HR_{est}^{(i)} - HR_{gt}^{(i)}|
$$
You can use MAE as your evaluation metric in your report. Other metrics are also welcomed.

In [199]:
video.shape

(1800, 80, 60, 3)

In [200]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"/Users/zihan/Documents/hku/aiot/group proj/rPPG-project/code/model/shape_predictor_81_face_landmarks.dat")  
 

In [201]:
MIN_HZ = 0.85  # 51bpm
MAX_HZ = 2.5 # 150bpm
BETA = 0.1 # smoothing factor

bpm_f = 60
bpm_l = 60
bpm_r = 60
bpm_avg = 60

predictions = []

In [202]:
# %pip uninstall -y obspy
# %pip install obspy

In [203]:
# %pip uninstall -y lxml
# %pip install lxml

In [204]:
from obspy.signal.detrend import polynomial

def Signal_Preprocessing(rgbsig):
        data = np.array(rgbsig)
        data_r = polynomial(data[:, 0], order=2)
        data_g = polynomial(data[:, 1], order=2)
        data_b = polynomial(data[:, 2], order=2)
        return np.array([data_r, data_g, data_b]).T

def Signal_Preprocessing_single(sig):
        return polynomial(sig, order=2)

def transfer2bmp(beta, _bpm, spec, fps):
        return beta * _bpm + (1 - beta) * np.argmax(spec[:int(len(spec)/2)])/len(spec) * fps * 60



In [205]:
# %pip install --upgrade lxml

In [206]:
def marker(frame):
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # cv.imshow('image-gray', frame_gray)
        # cv.waitKey(0) 
        # # closing all open windows 
        # cv.destroyAllWindows() 

        faces = detector(frame_gray)
        print('num of faces:', len(faces))
        if len(faces) == 1:
            face = faces[0]
            landmarks = [[p.x, p.y] for p in predictor(frame_gray, face).parts()]
        try:
            return landmarks
        except:
            return None
        
def roi(frame, flag_face):
    frame = cv.GaussianBlur(frame, (5, 5), 0)
    landmark = marker(frame)
    print('landmark:', landmark)
    cheek_l = [1, 2, 3, 4, 48, 31, 28, 39]
    cheek_r = [15, 14, 14, 12, 54, 35, 28, 42]
    forehead = [69, 70, 71, 80, 72, 25, 24, 23, 22, 21, 20, 19, 18]

    mask_l = np.zeros(frame.shape, dtype=np.uint8)
    print('mask size:', mask_l.shape)
    mask_r = np.zeros(frame.shape, dtype=np.uint8)
    mask_f = np.zeros(frame.shape, dtype=np.uint8)
    mask_display = np.zeros(frame.shape, dtype=np.uint8)
    try:
        print('No exception')
        flag_face = True
        pts_l = np.array(
            [landmark[i] for i in cheek_l], np.int32).reshape((-1, 1, 2))
        pts_r = np.array(
            [landmark[i] for i in cheek_r], np.int32).reshape((-1, 1, 2))
        pts_f = np.array(
            [landmark[i] for i in forehead], np.int32).reshape((-1, 1, 2))
        
        # print('pts_l:', pts_l)
        mask_l = cv.fillPoly(mask_l, [pts_l], (255, 255, 255))
        mask_r = cv.fillPoly(mask_r, [pts_r], (255, 255, 255))
        mask_f = cv.fillPoly(mask_f, [pts_f], (255, 255, 255))
        
        # for i in range(mask_l.shape[0]):
        #     for j in range(mask_l.shape[1]):
        #         for k in range(mask_l.shape[2]):
        #             print('mask value:', mask_l[i][j][k])
        #             if mask_l[i][j][k] != 0:
        #                 print("Not empty!!!!!!")
        #                 break
                    

        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15, 30))
        mask_l = cv.erode(mask_l, kernel, iterations=1)
        mask_r = cv.erode(mask_r, kernel, iterations=1)
        mask_f = cv.erode(mask_f, kernel, iterations=1)
        # print('mask_l:', mask_l)
        
        mask_display_l, mask_display_r, mask_display_f = copy.copy(mask_l), copy.copy(mask_r), copy.copy(mask_f)
        mask_display_l[:, :, 1] = 0
        mask_display_r[:, :, 0] = 0
        mask_display_f[:, :, 2] = 0

        mask_display = cv.bitwise_or(mask_display_l, mask_display_r)
        mask_display = cv.bitwise_or(mask_display, mask_display_f)
        face_mask = cv.addWeighted(mask_display, 0.25, frame, 1, 0)

        roi_l = cv.bitwise_and(mask_l, frame)
        roi_r = cv.bitwise_and(mask_r, frame)
        roi_f = cv.bitwise_and(mask_f, frame)

        return roi_l, roi_r, roi_f, flag_face, face_mask
    
    except Exception as e:
        print('Exception occurs')
        face_mask = frame
        flag_face = False
        return None, None, None, flag_face, face_mask

In [207]:
def RGB_hist(roi):
    b_hist = cv.calcHist([roi], [0], None, [256], [0, 256])
    g_hist = cv.calcHist([roi], [1], None, [256], [0, 256])
    r_hist = cv.calcHist([roi], [2], None, [256], [0, 256])

    b_hist = np.reshape(b_hist, (256))
    g_hist = np.reshape(g_hist, (256))
    r_hist = np.reshape(r_hist, (256))

    b_hist[0] = 0
    g_hist[0] = 0
    r_hist[0] = 0

    r_hist = r_hist/np.sum(r_hist)
    g_hist = g_hist/np.sum(g_hist)
    b_hist = b_hist/np.sum(b_hist)

    return [r_hist, g_hist, b_hist]

def Hist2Feature(hist):
    hist_r = hist[0]
    hist_g = hist[1]
    hist_b = hist[2]

    hist_r /= np.sum(hist_r)
    hist_g /= np.sum(hist_g)
    hist_b /= np.sum(hist_b)

    dens = np.arange(0, 256, 1)
    mean_r = dens.dot(hist_r)
    mean_g = dens.dot(hist_g)
    mean_b = dens.dot(hist_b)

    return [mean_r, mean_g, mean_b]


In [208]:
def CHROM(signal):
    X = signal
    Xcomp = 3*X[:, 0] - 2*X[:, 1]
    Ycomp = (1.5*X[:, 0])+X[:, 1]-(1.5*X[:, 2])
    sX = np.std(Xcomp)
    sY = np.std(Ycomp)
    alpha = sX/sY
    bvp = Xcomp - alpha * Ycomp
    return bvp

def butterworth_filter(data, low, high, sample_rate, order=5):
    nyquist_rate = sample_rate * 0.5
    low /= nyquist_rate
    high /= nyquist_rate
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.lfilter(b, a, data)

In [209]:
def cal_bpm(flag_queue, Sig_f, Sig_l, Sig_r, fps, bpm_avg, bpm_f, bpm_l, bpm_r):
    Sig_f = np.array(Sig_f)
    Sig_l = np.array(Sig_l)
    Sig_r = np.array(Sig_r)
    if flag_queue:
        if Sig_f.size != 1:
            # self.bvp_f_raw = self.processor.PBV(Sig_f)
            bvp_f_raw = CHROM(Sig_f)
            conf_f = 1 / \
                (max(bvp_f_raw) - min(bvp_f_raw))
            bvp_f = butterworth_filter(Signal_Preprocessing_single(bvp_f_raw), MIN_HZ, MAX_HZ, fps, order=5)
            spc_f = np.abs(np.fft.fft(bvp_f))
            bpm_f = transfer2bmp(BETA, bpm_f, spc_f, fps)
        if Sig_l.size != 1:
            # self.bvp_l_raw = self.processor.GREEN(Sig_l)
            bvp_l_raw = CHROM(Sig_l)
            conf_l = 1 / \
                (max(bvp_l_raw)-min(bvp_l_raw))
            bvp_l = butterworth_filter(Signal_Preprocessing_single(bvp_l_raw), MIN_HZ, MAX_HZ, fps, order=5)
            spc_l = np.abs(np.fft.fft(bvp_l))
            bpm_l = transfer2bmp(BETA, bpm_l, spc_l, fps)
        if Sig_r.size != 1:
            # self.bvp_r_raw = self.processor.CHROM(Sig_r)
            bvp_r_raw = CHROM(Sig_r)
            conf_r = 1 / \
                (max(bvp_r_raw)-min(bvp_r_raw))
            bvp_r = butterworth_filter(Signal_Preprocessing_single(bvp_r_raw), MIN_HZ, MAX_HZ, fps, order=5)
            spc_r = np.abs(np.fft.fft(bvp_r))
            bpm_r = transfer2bmp(BETA, bpm_r, spc_r, fps)
            
        total_conf = conf_f+conf_l+conf_r
        conf_f = conf_f/total_conf
        conf_l = conf_l/total_conf
        conf_r = conf_r/total_conf
        bpm_avg = bpm_f*conf_f+bpm_l * conf_l+bpm_r*conf_r

    return bpm_avg, bpm_f, bpm_l, bpm_r
    # else:
    #     return bpm_avg, bpm_f, bpm_l, bpm_r
    

In [210]:
fps = 30
QUEUE_MAX = 256
QUEUE_WINDOWS = 32
Queue_rawframe = Queue(maxsize=3)

Queue_Sig_l = Queue(maxsize=QUEUE_MAX)
Queue_Sig_r = Queue(maxsize=QUEUE_MAX)
Queue_Sig_f = Queue(maxsize=QUEUE_MAX)
Queue_Time = Queue(maxsize=QUEUE_WINDOWS)

flag_face, flag_queue = False, False  
Sig_l = None
Sig_r = None
Sig_f = None

face_mask = None
frame_display = None

for frame in video:
    # cv.imshow('image', frame)
    # cv.waitKey(0) 
    # # closing all open windows 
    # cv.destroyAllWindows() 
    # let's upscale the image using new  width and height
    up_width = 1080
    up_height = 1920
    up_points = (up_width, up_height)
    frame = cv.resize(frame, up_points, interpolation= cv.INTER_LINEAR)
    frame *= 255
    frame = frame.astype('uint8') 
    # cv.imshow('image-255', frame)
    # cv.waitKey(0) 
    # # closing all open windows 
    # cv.destroyAllWindows() 

    # print('original frame', frame)
    frame_display = copy.copy(frame)
    if Queue_Time.full():
        Queue_Time.get_nowait()
    if Queue_rawframe.full():
                Queue_rawframe.get_nowait()
    else:
        Queue_Time.put_nowait(time.time())
            
    try:
        Queue_rawframe.put_nowait(frame)
    except Exception as e:
        pass
    
    roi_l, roi_r, roi_f, flag_face, face_mask = roi(frame, flag_face)
    # print(roi_l)
    if roi_l is not None and roi_r is not None and roi_f is not None:
        hist_l = RGB_hist(roi_l)
        hist_r = RGB_hist(roi_r)
        hist_f = RGB_hist(roi_f)
        # print(self.Queue_signal_l.qsize(), self.Queue_signal_r.qsize(), self.Queue_signal_f.qsize())
        # print(self.flag_queue)
        if Queue_Sig_l.full():
            Sig_l = copy.copy(list(Queue_Sig_l.queue))
            Queue_Sig_l.get_nowait()
            # flag_queue = True
        else:
            flag_queue = False

        if Queue_Sig_r.full():
            Sig_r = copy.copy(list(Queue_Sig_r.queue))
            Queue_Sig_r.get_nowait()
            # flag_queue = True
        else:
            flag_queue = False
        
        if Queue_Sig_f.full():
            Sig_f = copy.copy(list(Queue_Sig_f.queue))
            Queue_Sig_f.get_nowait()
            flag_queue = True
        else:
            flag_queue = False

        Queue_Sig_l.put_nowait(Hist2Feature(hist_l))
        Queue_Sig_r.put_nowait(Hist2Feature(hist_r))
        Queue_Sig_f.put_nowait(Hist2Feature(hist_f))
    
    else:
        # print('Data is cleared')
        hist_l, hist_r, hist_f = None, None, None
        Queue_Sig_l.queue.clear()
        Queue_Sig_r.queue.clear()
        Queue_Sig_f.queue.clear()

    print(Queue_Sig_f.qsize())
    bpm_avg, bpm_f, bpm_l, bpm_r = cal_bpm(flag_queue, Sig_f, Sig_l, Sig_r, fps, bpm_avg, bpm_f, bpm_l, bpm_r)
    predictions.append(bpm_avg)

print(predictions)  


num of faces: 1
landmark: [[166, 670], [174, 761], [190, 847], [209, 932], [240, 1010], [290, 1076], [356, 1129], [429, 1171], [511, 1185], [594, 1175], [673, 1137], [744, 1087], [799, 1023], [834, 946], [856, 862], [875, 775], [884, 684], [227, 583], [267, 532], [331, 510], [401, 514], [466, 538], [583, 542], [649, 518], [719, 515], [785, 540], [825, 595], [522, 637], [521, 688], [521, 736], [520, 787], [458, 854], [489, 859], [519, 864], [550, 860], [582, 856], [302, 664], [340, 642], [388, 641], [428, 669], [385, 681], [337, 681], [617, 674], [660, 646], [707, 647], [747, 671], [708, 688], [659, 686], [405, 993], [449, 957], [489, 934], [517, 942], [544, 934], [583, 958], [624, 995], [583, 1024], [545, 1032], [516, 1034], [484, 1031], [447, 1021], [429, 991], [488, 973], [517, 975], [544, 974], [601, 993], [544, 983], [516, 984], [487, 982], [289, 351], [344, 315], [434, 319], [545, 321], [709, 329], [781, 379], [866, 545], [213, 468], [260, 384], [176, 657], [877, 671], [816, 430],

In [211]:
with np.printoptions(threshold=np.inf):
    print(gt)

[[125.          47.51835373  95.          77.77753059  85.51835373
  138.47942158  44.          62.          45.96329255  58.66629588
   59.88765295 122.55283648 136.33481646 169.13904338  78.
   73.7753059   60.51390434 126.29699666 153.38820912  74.13459399
  221.64380176 167.10989989 128.0807564  113.73073786 116.92070555
  110.69862811 111.87690026 125.80059325 132.11631972 119.50537634
   98.61423804  85.73793103  79.41957731  77.98509455  75.76040044
   73.46903967  72.24434557  71.          68.79495736  67.00719318
   65.27890248  63.05420838  61.76284761  60.20964034  58.
   57.          58.          57.          56.37287462  55.
   74.24390063 185.19428995 250.86929922 209.94697812 159.33036707
  127.46006674 124.73147942 130.18487208 118.20645161 116.57085651
  129.32658509 133.0153345  118.62068966  98.59844271  85.85873192
   79.72821654  77.          74.45463213  72.98746756  71.
   69.53807935  67.31338524  64.02202447  62.59466073  60.
   57.32619577  55.88445495  54.891

In [212]:
N = gt.shape[1]
gt_value = gt[0]
print(gt_value.shape)
print(len(predictions))
MAE = 0
for i in range(N):
    sig_gt = gt_value[i]
    sig_pred = predictions[i]
    MAE += abs(sig_pred - sig_gt)
MAE /= N
print(MAE)

(1800,)
1800
41.299940901908
